In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import librosa
import soundfile as sf
import time
import os

In [2]:
sampleSize=50000
sample_rate=16000
quantization_channels=256
dilations=[2**i for i in range(8)]*15
skipDim=256
residualDim=48
filterSize=3

In [3]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
#device = 'cpu'
torch.set_default_tensor_type('torch.cuda.FloatTensor')
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [5]:
def mu_law_encode(audio, quantization_channels=quantization_channels):
    '''Quantizes waveform amplitudes.'''
    mu = (quantization_channels - 1)*1.0
    # Perform mu-law companding transformation (ITU-T, 1988).
    # Minimum operation is here to deal with rare large amplitudes caused
    # by resampling.
    safe_audio_abs = np.minimum(np.abs(audio), 1.0)
    magnitude = np.log1p(mu * safe_audio_abs) / np.log1p(mu)
    signal = np.sign(audio) * magnitude
    # Quantize signal to the specified number of levels.
    #if(forX):return signal
    return ((signal + 1) / 2 * mu + 0.5).astype(int)
def mu_law_decode(output, quantization_channels=quantization_channels):
    '''Recovers waveform from quantized values.'''
    mu = quantization_channels - 1
    # Map values back to [-1, 1].
    signal = 2 * ((output*1.0) / mu) - 1
    # Perform inverse of mu-law transformation.
    magnitude = (1 / mu) * ((1 + mu)**np.abs(signal) - 1)
    return np.sign(signal) * magnitude

In [6]:
def readAudio(name):
    audio0, samplerate = sf.read(name, dtype='float32')
    return librosa.resample(audio0.T, samplerate, sample_rate).reshape(-1)
p=['./vsCorpus/origin_mix.wav','./vsCorpus/origin_vocal.wav',
   './vsCorpus/origin_mix.wav','./vsCorpus/origin_vocal.wav','./vsCorpus/pred_mix.wav']
xtrain,ytrain,xval,yval,xtest=readAudio(p[0]),readAudio(p[1]),readAudio(p[2]),readAudio(p[3]),readAudio(p[4])
assert((xtrain==xval).all())
assert((ytrain==yval).all())
assert((xtrain != ytrain).any())
#xtrain,ytrain,xval,yval=xtrain[:-sampleSize],ytrain[:-sampleSize],xval[-sampleSize:],yval[-sampleSize:]
xtrain,ytrain,xval,yval=xtrain[:-sampleSize],ytrain[:-sampleSize],xval[:sampleSize],yval[:sampleSize]
xtrain,xval,xtest=xtrain.reshape(1,1,-1),xval.reshape(1,1,-1),xtest.reshape(1,1,-1)
ytrain,yval=ytrain.reshape(1,-1),yval.reshape(1,-1)

In [7]:
xmean,xstd = xtrain.mean(),xtrain.std()
xtrain=(xtrain-xmean)/xstd
xval=(xval-xmean)/xstd
xtest=(xtest-xmean)/xstd
ytrain,yval=mu_law_encode(ytrain),mu_law_encode(yval)

In [8]:
xtrain,ytrain,xval,yval,xtest = torch.from_numpy(xtrain).type(torch.float32),\
                                torch.from_numpy(ytrain).type(torch.LongTensor),\
                                torch.from_numpy(xval).type(torch.float32),\
                                torch.from_numpy(yval).type(torch.LongTensor),\
                                torch.from_numpy(xtest).type(torch.float32)

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        sd,qd,rd = skipDim,quantization_channels,residualDim
        self.causal = nn.Conv1d(in_channels=1,out_channels=rd,kernel_size=3,padding=1)
        self.layer=dict()
        for i, d in enumerate(dilations):
            self.layer['tanh'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,padding=d,dilation=d)
            self.layer['sigmoid'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,padding=d,dilation=d)
            self.layer['skip'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=sd,kernel_size=1,padding=0)
            self.layer['dense'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=1,padding=0)
        self.post1 = nn.Conv1d(in_channels=sd,out_channels=sd,kernel_size=1,padding=0)
        self.post2 = nn.Conv1d(in_channels=sd,out_channels=qd,kernel_size=1,padding=0)
        self.tanh,self.sigmoid = nn.Tanh(),nn.Sigmoid()

    def forward(self, x):
        x = self.causal(x)
        #print('x.shape',x.shape)
        skip_connections = torch.zeros([1,skipDim,x.shape[2]],dtype=torch.float32,device=device)
        #skip_connections = []
        #print('skip_connections',skip_connections.shape)
        for i, dilation in enumerate(dilations):
            xinput = x.clone()
            x1 = self.tanh(self.layer['tanh'+str(i)](x))
            #print('tanh.shape',x1.shape)
            x2 = self.sigmoid(self.layer['sigmoid'+str(i)](x))
            #print('sigmoid.shape',x2.shape)
            x = x1*x2
            #print('multi',x3.shape)
            skip_connections += self.layer['skip'+str(i)](x)
            #print(skip_connections.cpu()[:,:,:10])
            #cur =self.layer['skip'+str(i)](x3)
            #skip_connections.append(cur)
            #print('skip.shape',self.layer['skip'+str(i)](x).shape)
            x = self.layer['dense'+str(i)](x)
            #print('dense.shape',x.shape)
            x += xinput
        #skip_connections = torch.cat(skip_connections,dim=0)
        #skip_connections = torch.sum(skip_connections,dim=0,keepdim=True)
        x = self.post2(F.relu(self.post1(F.relu(skip_connections))))
        return x

model = Net().cuda()
criterion = nn.CrossEntropyLoss().cuda()
#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer = optim.Adam(model.parameters(),weight_decay=1e-5)

In [10]:
#torch.sum(c,dim=0,keepdim=True)

In [11]:
#model = torch.load('torchmodel0.681600')

In [14]:
def val():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        #data, target = xval.to(device), yval.to(device)
        data, target = xtrain[:,:,:176000].to(device), ytrain[:,:176000].to(device)
        output = model(data)
        #print(output.shape)
        #print(output[:,:,:10])
        val_loss = criterion(output, target).item()
    print('\nval set:  {:.4f}, ({:.3f} sec/step)\n'.format(val_loss,time.time()-startval_time))

def test():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        output = model(xtest.to(device))
        pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
        ans = mu_law_decode(pred)
        sf.write('./vsCorpus/resultxte.wav', ans, sample_rate)
        
        #output = model(xtrain[:,:,:sampleSize].to(device))
        output = model(xtrain[:,:,:176000].to(device))
        pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
        ans = mu_law_decode(pred)
        sf.write('./vsCorpus/resultxtr.wav', ans, sample_rate)
        print('stored done\n')
    
def train(epoch):
    model.train()
    #idx = np.arange(xtrain.shape[-1] - 2 * sampleSize,1000)
    #176000
    idx = np.arange(0,126000,3000)
    np.random.shuffle(idx)
    for i, ind in enumerate(idx):
        start_time = time.time()
        data, target = xtrain[:,:,ind:ind+sampleSize].to(device), ytrain[:,ind:ind+sampleSize].to(device)
        output = model(data)
        #print(output.shape)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)] Loss: {:.6f} , ({:.3f} sec/step)'.format(
                epoch, i, len(idx),100. * i / len(idx), loss.item(),time.time() - start_time))
        if i % 100 == 0:
            val()
            test()
            torch.save(model, '15*2**8')

In [15]:
for epoch in range(100000):
    train(epoch)
    test()

Train Epoch: 0 [0/42 (0%)] Loss: 3.724951 , (2.942 sec/step)

val set:  3.9157, (4.142 sec/step)

stored done



/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Train Epoch: 0 [1/42 (2%)] Loss: 3.837363 , (2.941 sec/step)
Train Epoch: 0 [2/42 (5%)] Loss: 3.607630 , (2.944 sec/step)
Train Epoch: 0 [3/42 (7%)] Loss: 3.681419 , (2.939 sec/step)
Train Epoch: 0 [4/42 (10%)] Loss: 3.592268 , (2.944 sec/step)
Train Epoch: 0 [5/42 (12%)] Loss: 3.650419 , (2.940 sec/step)
Train Epoch: 0 [6/42 (14%)] Loss: 3.607689 , (2.945 sec/step)
Train Epoch: 0 [7/42 (17%)] Loss: 3.760202 , (2.941 sec/step)
Train Epoch: 0 [8/42 (19%)] Loss: 3.582067 , (2.946 sec/step)
Train Epoch: 0 [9/42 (21%)] Loss: 3.612819 , (2.942 sec/step)
Train Epoch: 0 [10/42 (24%)] Loss: 3.652275 , (2.945 sec/step)
Train Epoch: 0 [11/42 (26%)] Loss: 3.564987 , (2.943 sec/step)
Train Epoch: 0 [12/42 (29%)] Loss: 4.161965 , (2.946 sec/step)
Train Epoch: 0 [13/42 (31%)] Loss: 4.053448 , (2.942 sec/step)
Train Epoch: 0 [14/42 (33%)] Loss: 3.512703 , (2.945 sec/step)
Train Epoch: 0 [15/42 (36%)] Loss: 3.859648 , (2.942 sec/step)
Train Epoch: 0 [16/42 (38%)] Loss: 3.981323 , (2.945 sec/step)
Trai

Train Epoch: 3 [3/42 (7%)] Loss: 3.431967 , (2.941 sec/step)
Train Epoch: 3 [4/42 (10%)] Loss: 3.509096 , (2.944 sec/step)
Train Epoch: 3 [5/42 (12%)] Loss: 3.548517 , (2.941 sec/step)
Train Epoch: 3 [6/42 (14%)] Loss: 3.867289 , (2.945 sec/step)
Train Epoch: 3 [7/42 (17%)] Loss: 3.428515 , (2.941 sec/step)
Train Epoch: 3 [8/42 (19%)] Loss: 3.655333 , (2.945 sec/step)
Train Epoch: 3 [9/42 (21%)] Loss: 3.462684 , (2.941 sec/step)
Train Epoch: 3 [10/42 (24%)] Loss: 4.076271 , (2.944 sec/step)
Train Epoch: 3 [11/42 (26%)] Loss: 3.379539 , (2.942 sec/step)
Train Epoch: 3 [12/42 (29%)] Loss: 3.979886 , (2.944 sec/step)
Train Epoch: 3 [13/42 (31%)] Loss: 3.474401 , (2.941 sec/step)
Train Epoch: 3 [14/42 (33%)] Loss: 3.551818 , (2.945 sec/step)
Train Epoch: 3 [15/42 (36%)] Loss: 3.346160 , (2.940 sec/step)
Train Epoch: 3 [16/42 (38%)] Loss: 3.549409 , (2.945 sec/step)
Train Epoch: 3 [17/42 (40%)] Loss: 3.566915 , (2.940 sec/step)
Train Epoch: 3 [18/42 (43%)] Loss: 3.827258 , (2.944 sec/step)


Train Epoch: 6 [5/42 (12%)] Loss: 3.419673 , (2.944 sec/step)
Train Epoch: 6 [6/42 (14%)] Loss: 3.532477 , (2.948 sec/step)
Train Epoch: 6 [7/42 (17%)] Loss: 4.152207 , (2.943 sec/step)
Train Epoch: 6 [8/42 (19%)] Loss: 3.732512 , (2.948 sec/step)
Train Epoch: 6 [9/42 (21%)] Loss: 3.412502 , (2.944 sec/step)
Train Epoch: 6 [10/42 (24%)] Loss: 3.371483 , (2.948 sec/step)
Train Epoch: 6 [11/42 (26%)] Loss: 3.370050 , (2.945 sec/step)
Train Epoch: 6 [12/42 (29%)] Loss: 3.824945 , (2.949 sec/step)
Train Epoch: 6 [13/42 (31%)] Loss: 3.285482 , (2.944 sec/step)
Train Epoch: 6 [14/42 (33%)] Loss: 3.411977 , (2.948 sec/step)
Train Epoch: 6 [15/42 (36%)] Loss: 3.348690 , (2.944 sec/step)
Train Epoch: 6 [16/42 (38%)] Loss: 3.204310 , (2.948 sec/step)
Train Epoch: 6 [17/42 (40%)] Loss: 3.693000 , (2.945 sec/step)
Train Epoch: 6 [18/42 (43%)] Loss: 3.536199 , (2.947 sec/step)
Train Epoch: 6 [19/42 (45%)] Loss: 3.458471 , (2.944 sec/step)
Train Epoch: 6 [20/42 (48%)] Loss: 3.252186 , (2.948 sec/ste

...

Train Epoch: 208 [21/42 (50%)] Loss: 2.299103 , (2.942 sec/step)
Train Epoch: 208 [22/42 (52%)] Loss: 2.941335 , (2.946 sec/step)
Train Epoch: 208 [23/42 (55%)] Loss: 2.801486 , (2.943 sec/step)
Train Epoch: 208 [24/42 (57%)] Loss: 1.936339 , (2.947 sec/step)
Train Epoch: 208 [25/42 (60%)] Loss: 1.742978 , (2.944 sec/step)
Train Epoch: 208 [26/42 (62%)] Loss: 1.798736 , (2.946 sec/step)
Train Epoch: 208 [27/42 (64%)] Loss: 1.764519 , (2.945 sec/step)
Train Epoch: 208 [28/42 (67%)] Loss: 1.554103 , (2.947 sec/step)
Train Epoch: 208 [29/42 (69%)] Loss: 1.861142 , (2.942 sec/step)
Train Epoch: 208 [30/42 (71%)] Loss: 1.906691 , (2.946 sec/step)
Train Epoch: 208 [31/42 (74%)] Loss: 1.861083 , (2.942 sec/step)
Train Epoch: 208 [32/42 (76%)] Loss: 1.589881 , (2.946 sec/step)
Train Epoch: 208 [33/42 (79%)] Loss: 2.100102 , (2.941 sec/step)
Train Epoch: 208 [34/42 (81%)] Loss: 3.729324 , (2.946 sec/step)
Train Epoch: 208 [35/42 (83%)] Loss: 2.172201 , (2.942 sec/step)
Train Epoch: 208 [36/42 (

Train Epoch: 211 [19/42 (45%)] Loss: 1.712608 , (2.944 sec/step)
Train Epoch: 211 [20/42 (48%)] Loss: 2.538886 , (2.948 sec/step)
Train Epoch: 211 [21/42 (50%)] Loss: 2.839251 , (2.945 sec/step)
Train Epoch: 211 [22/42 (52%)] Loss: 1.689397 , (2.948 sec/step)
Train Epoch: 211 [23/42 (55%)] Loss: 1.948068 , (2.945 sec/step)
Train Epoch: 211 [24/42 (57%)] Loss: 3.757761 , (2.947 sec/step)
Train Epoch: 211 [25/42 (60%)] Loss: 3.374318 , (2.943 sec/step)
Train Epoch: 211 [26/42 (62%)] Loss: 3.166953 , (2.945 sec/step)
Train Epoch: 211 [27/42 (64%)] Loss: 2.289276 , (2.943 sec/step)
Train Epoch: 211 [28/42 (67%)] Loss: 2.940922 , (2.949 sec/step)
Train Epoch: 211 [29/42 (69%)] Loss: 1.738003 , (2.946 sec/step)
Train Epoch: 211 [30/42 (71%)] Loss: 1.725588 , (2.949 sec/step)
Train Epoch: 211 [31/42 (74%)] Loss: 2.112353 , (2.945 sec/step)
Train Epoch: 211 [32/42 (76%)] Loss: 2.590670 , (2.952 sec/step)
Train Epoch: 211 [33/42 (79%)] Loss: 1.892278 , (2.944 sec/step)
Train Epoch: 211 [34/42 (

Train Epoch: 214 [17/42 (40%)] Loss: 1.583016 , (2.942 sec/step)
Train Epoch: 214 [18/42 (43%)] Loss: 1.870831 , (2.946 sec/step)
Train Epoch: 214 [19/42 (45%)] Loss: 1.670369 , (2.942 sec/step)
Train Epoch: 214 [20/42 (48%)] Loss: 3.403217 , (2.945 sec/step)
Train Epoch: 214 [21/42 (50%)] Loss: 1.630497 , (2.944 sec/step)
Train Epoch: 214 [22/42 (52%)] Loss: 1.877676 , (2.947 sec/step)
Train Epoch: 214 [23/42 (55%)] Loss: 2.271109 , (2.944 sec/step)
Train Epoch: 214 [24/42 (57%)] Loss: 1.777148 , (2.947 sec/step)
Train Epoch: 214 [25/42 (60%)] Loss: 3.146682 , (2.944 sec/step)
Train Epoch: 214 [26/42 (62%)] Loss: 2.130823 , (2.947 sec/step)
Train Epoch: 214 [27/42 (64%)] Loss: 3.747847 , (2.943 sec/step)
Train Epoch: 214 [28/42 (67%)] Loss: 1.882766 , (2.948 sec/step)
Train Epoch: 214 [29/42 (69%)] Loss: 2.126367 , (2.944 sec/step)
Train Epoch: 214 [30/42 (71%)] Loss: 1.613129 , (2.948 sec/step)
Train Epoch: 214 [31/42 (74%)] Loss: 1.703640 , (2.943 sec/step)
Train Epoch: 214 [32/42 (

Train Epoch: 217 [15/42 (36%)] Loss: 1.880903 , (2.946 sec/step)
Train Epoch: 217 [16/42 (38%)] Loss: 2.587484 , (2.949 sec/step)
Train Epoch: 217 [17/42 (40%)] Loss: 3.399376 , (2.945 sec/step)
Train Epoch: 217 [18/42 (43%)] Loss: 1.811038 , (2.949 sec/step)
Train Epoch: 217 [19/42 (45%)] Loss: 1.798299 , (2.944 sec/step)
Train Epoch: 217 [20/42 (48%)] Loss: 1.781963 , (2.950 sec/step)
Train Epoch: 217 [21/42 (50%)] Loss: 2.262394 , (2.945 sec/step)
Train Epoch: 217 [22/42 (52%)] Loss: 2.901592 , (2.949 sec/step)
Train Epoch: 217 [23/42 (55%)] Loss: 1.753457 , (2.945 sec/step)
Train Epoch: 217 [24/42 (57%)] Loss: 1.855883 , (2.949 sec/step)
Train Epoch: 217 [25/42 (60%)] Loss: 2.123746 , (2.947 sec/step)
Train Epoch: 217 [26/42 (62%)] Loss: 1.783709 , (2.950 sec/step)


KeyboardInterrupt: 

In [ ]:
#model = torch.load('torchmodel')

In [17]:
torch.save(model, 'loss23*1.+4*3.+16*2.step_9072_repeat15*2**8resu32sample50000')

/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
torch.

In [16]:
model

Net(
  (causal): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (post1): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
  (post2): Conv1d(512, 256, kernel_size=(1,), stride=(1,))
  (tanh): Tanh()
  (sigmoid): Sigmoid()
)